In [1]:
import os
import fnmatch
import pandas as pd
import numpy as np

In [2]:
path = '/home/ke/Desktop/phase_contrast/'

In [58]:
# for direct in os.listdir(path):
for direct in next(os.walk(path))[1]:
    save_path = '_'.join(direct.split(' ')).split('_')[0]+'_pc.csv'
    row_list = []
    if fnmatch.fnmatch(direct, 'Caro*') or fnmatch.fnmatch(direct, 'Vert*'):
        for file in os.listdir(path+direct):
            print(file)
            f_path = path+direct+'/'+file
            locs = {}
            tr = None
            with open(f_path, 'rb') as fp:
                for i, line in enumerate(fp):
                    if i == 52:
                        # 53th line
                        tr = str(line).split()[2]
                        print(tr)
                    elif i > 52:
                        if str(line)[2:5] == 'ROI':
#                             print(i, line)
                            label = line.decode('ascii').split(': ')[1][:-1].lower()
#                             print(label)
                            if label not in locs:
                                locs[label] = [i]
                            else:
                                locs[label].append(i)
            print(locs)
            for i in locs:
                f_start = locs[i][0]+1
                first = pd.read_csv(f_path, header=None, sep=':', skipinitialspace=True, 
                                    skiprows=f_start, nrows=4, engine='python').T
                first.columns = ['Avg_flow [cm/s]', 'HB_interv [ms]', 'Flow/HB [ml/hb]', 'Flow/min [ml/min]']
                first = first.drop(first.index[0]).reset_index(drop=True)
                first.loc[0] = first.loc[0].str.split(' ').str.get(0)
#                 print(first)
                head = ['Index', 'Phase Trigger', 'Area', 'Area_p', 'Vel', 'SD_Vel', 
                        'Min_V', 'Max_V', 'Flow', 'Sum']
                
                s_start = locs[i][1]+2
                second = pd.read_csv(f_path, names=head, sep=' +',
                                     skiprows=s_start, nrows=30, engine='python',
                                     usecols=['Max_V', 'Min_V'])
#                 print(second)
                pi = np.round((second.Max_V.max() - second.Min_V.min())/float(first['Avg_flow [cm/s]'][0]), 4)
                ri = np.round((second.Max_V.max() - second.Min_V.min())/second.Max_V.max(), 4)
                temp = {'TR [ms]' : [tr],
                        'PI' : [pi],
                        'RI' : [ri]}
                df = pd.DataFrame(temp)
                df = pd.concat((first, df), axis = 1)
                df['Patient'] = file.split('_')[0]
                print(df)
                row_list.append(df)
                            
            print()
        continue
    else:
    #     print(direct)
    #     print(file.split(' '))
    #     print('_'.join(file.split(' ')))
    #     print('_'.join(file.split(' ')).split('_'))
    #     print('_'.join(file.split(' ')).split('_')[0]+'_pc')
    #     df = pd.DataFrame(columns=['Patient', 'Area', 'V_min', 'V_max', 'Flow_min', 'Flow_max', 'Flow_total'])
        header = ['Patient', 'Area [cm^2]', 'V_min [cm/s]', 'V_max [cm/s]',
                  'Flow_min [ml/s]', 'Flow_max [ml/s]', 'Flow_pos [ml/s]', 'Flow_neg [ml/s]']
        for file in os.listdir(path+direct):
    #         print(file.split('_')[0])
            patient = file.split('_')[0]
            head = ['Index', 'Phase Trigger', 'Area', 'Area_p', 'Vel', 'SD_Vel', 
                    'Min_V', 'Max_V', 'Flow', 'Sum']
            f_path = path+direct+'/'+file
            with open(f_path, 'rb') as fp:
                num_lines = sum(1 for line in fp)
    #         print(num_lines)
            data = pd.read_csv(f_path, names=head,  skiprows=num_lines-33, sep=' +', engine='python',
                               usecols=['Area', 'Vel', 'Min_V', 'Max_V', 'Flow'])
            data.Area = data.Area / 100
    #         info = [patient, np.round(data.Area[0], 4), data.Min_V.min(), data.Max_V.max(),
    #                  data.Flow.min(), data.Flow.max(), np.round(np.sum(data.Vel * data.Area[0]), 4)]
            info = {'Patient' : patient, 
                    'Area [cm^2]' : np.round(data.Area[0], 4), 
                    'V_min [cm/s]' : data.Min_V.min(), 
                    'V_max [cm/s]' : data.Max_V.max(),
                    'Flow_min [ml/s]' : data.Flow.min(), 
                    'Flow_max [ml/s]' : data.Flow.max(),
                    'Flow_pos [ml/s]' : np.round(np.sum(data.Vel[data.Vel > 0] * data.Area[0]), 4),
                    'Flow_neg [ml/s]' : np.round(np.sum(data.Vel[data.Vel < 0] * data.Area[0]), 4)}
    #         df = df.append(info)
    #         df = df.append(info, ignore_index=True)
            row_list.append(info)
        final = pd.DataFrame(row_list, columns=header).sort_values('Patient').reset_index(drop=True)
#         print(final)
#         final.to_csv(path+save_path, sep='\t', index=False)
        print('\n')

WH1340_PC_verts.txt
33.31
{'rt vert': [63, 81], 'lt vert': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -5.21         999.31           -1.60            -95.82   33.31   

       PI      RI Patient  
0 -6.5374  2.4259  WH1340  
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -5.72         999.31           -1.81           -108.59   33.31   

       PI      RI Patient  
0 -6.2762  3.5686  WH1340  

EX1109_PC_verts.txt
30.76
{'rt vert': [63, 81], 'lt vert': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -6.41         922.76           -1.25            -81.08   30.76   

       PI      RI Patient  
0 -7.2434  2.5652  EX1109  
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -9.06         922.76           -1.38            -89.56   30.76   

       PI      RI Patient  
0 -6.0243  2.3445  EX1109  

SH002_

  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -6.43         999.31           -1.40            -84.22   33.31   

        PI     RI Patient  
0 -27.4075  1.877  WH1372  

EX38_PC_verts.txt
42.55
{'rt vertebral artery': [63, 81], 'lt vertebral artery': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -2.50       1,276.55           -0.38            -17.97   42.55   

      PI      RI Patient  
0 -12.16  1.9119    EX38  
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -4.25       1,276.55           -0.73            -34.35   42.55   

       PI      RI Patient  
0 -9.7906  2.1176    EX38  

EX46_PC_verts.txt
38.45
{'rt vert': [63, 81], 'lt vert': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -3.45       1,153.45           -1.17            -60.69   38.45   

        PI      RI Patient  
0 -10.2551  3.0

  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0          -12.25         844.14           -2.25           -160.28   28.14   

       PI      RI Patient  
0 -3.2441  5.2706   BR031  

WH1708_PC_verts.txt
36.34
{'rt vert': [63, 81], 'lt vert': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -6.86       1,090.34           -1.77            -97.23   36.34   

       PI      RI Patient  
0 -6.0539  3.1415  WH1708  
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -6.68       1,090.34           -2.42           -133.04   36.34   

       PI     RI Patient  
0 -8.0225  2.604  WH1708  

WH1240_PC_verts.txt
24.07
{'rt vert': [63, 81], 'lt vert': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0           -6.99         722.07           -0.89            -73.63   24.07   

       PI      RI Patient  
0 -5.3133  2.3566  WH1240  
  Avg_fl

  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0          -12.88         844.14           -2.73           -194.04   28.14   

      PI      RI Patient  
0 -2.597  7.1781   BR031  

EX46_PC_carotids.txt
38.45
{'rt ica': [63, 81], 'lt ica': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0          -11.33       1,153.45           -3.67           -191.02   38.45   

       PI      RI Patient  
0 -3.6893  6.2295    EX46  
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0          -10.62       1,153.45           -3.77           -196.22   38.45   

       PI     RI Patient  
0 -4.3126  3.782    EX46  

EX41_PC_carotids.txt
33.31
{'rt ica': [63, 81], 'lt ica': [70, 115]}
  Avg_flow [cm/s] HB_interv [ms] Flow/HB [ml/hb] Flow/min [ml/min] TR [ms]  \
0          -17.23         999.31           -6.49           -389.41   33.31   

       PI      RI Patient  
0 -4.1207  4.5108    EX41  
  Avg_flow [

KeyError: 0